In [1]:
import pymongo
from bson.json_util import dumps, loads 
import pandas as pd
import numpy as np
from datetime import datetime
import os.path

# gcalendar api
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


In [2]:
path = '../event_site_project_access/mongodb_access.txt'
f = open(path, 'r')
mongo_access = f.read()
f.close()

#### Add an event to MongoDB

In [11]:
# pymongo setting
path = '../event_site_project_access/mongodb_access.txt'
f = open(path, 'r')
mongo_access = f.read()
f.close()
client = pymongo.MongoClient(mongo_access)

db = client['event_site']
# event_info_collection
event_info_collection = db['event_information']

# event_info_collection
space_info_collection = db['space_information']


In [3]:
# gcalendar setting

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/calendar']
'''
creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('../event_site_project_access/token.json'):
    creds = Credentials.from_authorized_user_file('../event_site_project_access/token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    #if creds and creds.expired and creds.refresh_token:
    #    creds.refresh(Request())
    #else:
    flow = InstalledAppFlow.from_client_secrets_file('../event_site_project_access/credentials.json', SCOPES)
    creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('../event_site_project_access/token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('calendar', 'v3', credentials = creds)
except HttpError as error:
    print('An error occurred: %s' % error)
'''

creds = service_account.Credentials.from_service_account_file(
    '../event_site_project_access/service_account_credentials.json',
    scopes = SCOPES
)

try:
    service = build('calendar', 'v3', credentials = creds)
except HttpError as error:
    print('An error occurred: %s' % error)

# 需提供權限給 service account

In [4]:
service.close()

In [4]:
calendarId = '62c2d3e83153931e66c3b91b3edcecad88eb8fa2c0df6f8a1fa74d6d765c16be@group.calendar.google.com'

In [ ]:
# https://developers.google.com/calendar/api/v3/reference/events/get

In [ ]:
def build_event(**kwargs): 
    '''
    Input the collection and kwargs to building events in the target DB and google calendar. 
    - kwargs: event information dictionary which includes event_name, event_bgn_time, event_end_time, event_location, event_host
    '''
    # create a dict to put event info into google calendar
    gevent_info = {
    'summary': kwargs['event_name'], 
    'location': kwargs['event_location'], 
    'start': {
        'dateTime': kwargs['event_bgn_time'].isoformat('T'), 
        'timeZone': 'Asia/Taipei', 
    }, 
    'end': {
        'dateTime': kwargs['event_end_time'].isoformat('T'), 
        'timeZone': 'Asia/Taipei', 
    }, 
    'reminders': {
        'useDefault': False, 
        'overrides': [
        {'method': 'email', 'minutes': 24 * 60},
        {'method': 'popup', 'minutes': 10},
        ],
    }, 
    }
    event = service.events().insert(calendarId = calendarId, body = gevent_info).execute()
    print('Event created: %s' % (event.get('htmlLink')), ', ', 'id: %s' % (event.get('id')))

    gcalendar_id = event.get('id')

    # create a dict to put event info into DB
    try: 
        max_event_id = event_info_collection.find_one(sort = [('event_id', -1)])['event_id']
    except:
        max_event_id = 0
    info = {'event_id': max_event_id + 1, 'gcalendar_id': gcalendar_id}
    for k, v in kwargs.items(): 
        info[k] = v
    info['updt_date'] = datetime.now()
    event_info_collection.insert_many([info])
    print('Event inserted to mongodb. ')

In [25]:
def build_event(**kwargs): 
    '''
    Input the collection and kwargs to building events in the target DB and google calendar. 
    - kwargs: event information dictionary which includes event_name, event_bgn_time, event_end_time, event_location, event_host
    '''
    # create a dict to put event info into google calendar
    gevent_info = {
    'summary': kwargs['event_name'], 
    'location': kwargs['event_location'], 
    'start': {
        'dateTime': kwargs['event_bgn_time'].isoformat('T'), 
        'timeZone': 'Asia/Taipei', 
    }, 
    'end': {
        'dateTime': kwargs['event_end_time'].isoformat('T'), 
        'timeZone': 'Asia/Taipei', 
    }, 
    'reminders': {
        'useDefault': False, 
        'overrides': [
        {'method': 'email', 'minutes': 24 * 60},
        {'method': 'popup', 'minutes': 10},
        ],
    }, 
    }
    event = service.events().insert(calendarId = calendarId, body = gevent_info).execute()
    print('Event created: %s' % (event.get('htmlLink')), ', ', 'id: %s' % (event.get('id')))

    gcalendar_id = event.get('id')

    # create a dict to put event info into DB
    try: 
        max_event_id = event_info_collection.find_one(sort = [('event_id', -1)])['event_id']
    except:
        max_event_id = 0
    event_id_allocate = max_event_id + 1
    info = {'event_id': event_id_allocate, 'gcalendar_id': gcalendar_id}
    for k, v in kwargs.items(): 
        info[k] = v
    info['updt_date'] = datetime.now()
    event_info_collection.insert_many([info])
    print('Event inserted to mongodb. ')

    # event_tag
    if kwargs.get('tag'):
        for tag in kwargs['tag']:
            if event_tag_collection.find({'tag': tag}).distinct('event_id'):
                event_tag_collection.update_one({'tag': tag}, {'$push': {'event_id': event_id_allocate}})
            else:
                event_tag_collection.insert_one({'tag': tag, 'event_id': [event_id_allocate]})

In [71]:
event_name = 'ABC workshop'
event_bgn_time = datetime.fromisoformat('2023-10-07T12:00:00') 
event_end_time = datetime.fromisoformat('2023-10-07T14:00:00') 
event_location = 'A01'
event_host = 's711033123@gm.ntpu.edu.tw'

event_info = {
    'event_name': event_name, 
    'event_bgn_time': event_bgn_time, 
    'event_end_time': event_end_time, 
    'event_location': event_location, 
    'event_host': event_host, 
}

build_event(**event_info)

Event created: https://www.google.com/calendar/event?eid=OGE3MnVla3ZrYmpqNzRxcXQ2ZTA5OTRjNTAgNjJjMmQzZTgzMTUzOTMxZTY2YzNiOTFiM2VkY2VjYWQ4OGViOGZhMmMwZGY2ZjhhMWZhNzRkNmQ3NjVjMTZiZUBn ,  id: 8a72uekvkbjj74qqt6e0994c50
Event inserted to mongodb. 


In [33]:
event_name = 'K monthly meeting'
event_bgn_time = datetime.fromisoformat('2024-02-08T15:00:00') 
event_end_time = datetime.fromisoformat('2024-02-08T16:00:00') 
event_location = 'A05'
event_host = 's711033123@gm.ntpu.edu.tw'

event_info = {
    'event_name': event_name, 
    'event_bgn_time': event_bgn_time, 
    'event_end_time': event_end_time, 
    'event_location': event_location, 
    'event_host': event_host, 
}

build_event(**event_info)

Event created: https://www.google.com/calendar/event?eid=cGw1aGtzNXE3cDJwMDgyY2x1bDExZTllNnMgNjJjMmQzZTgzMTUzOTMxZTY2YzNiOTFiM2VkY2VjYWQ4OGViOGZhMmMwZGY2ZjhhMWZhNzRkNmQ3NjVjMTZiZUBn ,  id: pl5hks5q7p2p082clul11e9e6s
Event inserted to mongodb. 


In [34]:
event_name = 'Happy year-end party!!'
event_bgn_time = datetime.fromisoformat('2024-02-09T18:00:00') 
event_end_time = datetime.fromisoformat('2024-02-09T21:00:00') 
event_location = 'B01'
event_host = 's711033123@gm.ntpu.edu.tw'

event_info = {
    'event_name': event_name, 
    'event_bgn_time': event_bgn_time, 
    'event_end_time': event_end_time, 
    'event_location': event_location, 
    'event_host': event_host, 
}

build_event(**event_info)

Event created: https://www.google.com/calendar/event?eid=aDA2YjVyNGc3YTk5Y2hiZ3FtbjU1Y2tlNnMgNjJjMmQzZTgzMTUzOTMxZTY2YzNiOTFiM2VkY2VjYWQ4OGViOGZhMmMwZGY2ZjhhMWZhNzRkNmQ3NjVjMTZiZUBn ,  id: h06b5r4g7a99chbgqmn55cke6s
Event inserted to mongodb. 


In [38]:
event_name = 'MM employee training'
event_bgn_time = datetime.fromisoformat('2024-02-12T09:00:00') 
event_end_time = datetime.fromisoformat('2024-02-12T17:00:00') 
event_location = 'B02'
event_host = 's711033123@gm.ntpu.edu.tw'

event_info = {
    'event_name': event_name, 
    'event_bgn_time': event_bgn_time, 
    'event_end_time': event_end_time, 
    'event_location': event_location, 
    'event_host': event_host, 
}

build_event(**event_info)

Event created: https://www.google.com/calendar/event?eid=MzllaG9hMDljbmRzbW05NHM0bjBvaWFtdDQgNjJjMmQzZTgzMTUzOTMxZTY2YzNiOTFiM2VkY2VjYWQ4OGViOGZhMmMwZGY2ZjhhMWZhNzRkNmQ3NjVjMTZiZUBn ,  id: 39ehoa09cndsmm94s4n0oiamt4
Event inserted to mongodb. 


#### shows event in days range

In [68]:
range_bgn_time = datetime(2023, 10, 1, 0) 
range_end_time = datetime(2023, 10, 7, 0)

range_event = list(event_info_collection.find({'$and': [{
    'event_end_time': {'$gt': range_bgn_time}, 
    'event_bgn_time': {'$lt': range_end_time}
    }]}))


In [71]:
dumps(range_event)

'[{"_id": {"$oid": "6522c938f742e74f7c193998"}, "event_id": 2, "gcalendar_id": "65s73bi8fomivv4aok5rpqa4n0", "event_name": "GEM team party", "event_bgn_time": {"$date": 1696615200000}, "event_end_time": {"$date": 1696626000000}, "event_location": "A01", "event_host": "s711033123@gm.ntpu.edu.tw", "updt_date": {"$date": 1696807352727}}]'

#### 邀請使用者參加活動(add attendees)

In [83]:
# https://developers.google.com/calendar/api/concepts/inviting-attendees-to-events?hl=zh-tw
# https://developers.google.com/calendar/api/v3/reference/events/update

def add_attendees(gcalendar_id, attendee_email):
    # get the target event in google calendar
    event = service.events().get(calendarId = calendarId, eventId = gcalendar_id).execute()

    # get the current attendees list and add new attendee
    try:
        attendees_list = event['attendees']
    except: 
        attendees_list = []
    attendees_list.append({'email': attendee_email})

    # update the attendees list
    event['attendees'] = attendees_list
    updated_event = service.events().update(calendarId = calendarId, eventId = event['id'], body = event).execute()

    # print the new attendees list
    get_updated_event = service.events().get(calendarId = calendarId, eventId = event['id']).execute()
    print('Update attendees, new attendees list became ', get_updated_event['attendees'])

In [87]:
target_event = event_info_collection.find_one({'event_id': 1})
gcalendar_id = target_event['gcalendar_id']

add_attendees(gcalendar_id, 'chaoyunshan0@gmail.com')

Update attendees, new attendees list became  [{'email': 'chaoyunshan0@gmail.com', 'responseStatus': 'needsAction'}, {'email': 's711033123@gm.ntpu.edu.tw', 'responseStatus': 'needsAction'}]


In [88]:
target_event = event_info_collection.find_one({'event_id': 1})
gcalendar_id = target_event['gcalendar_id']

add_attendees(gcalendar_id, 's711033123@gm.ntpu.edu.tw')

Update attendees, new attendees list became  [{'email': 'chaoyunshan0@gmail.com', 'responseStatus': 'needsAction'}, {'email': 's711033123@gm.ntpu.edu.tw', 'responseStatus': 'needsAction'}]


In [90]:
# list out all event in a calendar
page_token = None
ans = service.events().list(calendarId = calendarId, pageToken = page_token).execute()
len(ans['items'])

2

#### 參加活動

In [ ]:
attendee_name = 'Vivian Shang'
attendee_email = 'alionking821@gmail.com'

# add attendee

# add attendance data into mongodb

### 建立場地

In [10]:
def build_space(**kwargs): 
    '''
    Input the collection and kwargs to building events space in the target DB. 
    - kwargs: space_name, space_capacity, type
    '''
    # create a dict to put info into DB
    try: 
        max_space_id = space_info_collection.find_one(sort = [('space_id', -1)])['space_id']
    except:
        max_space_id = 0

    space_info = {
    'space_id': max_space_id + 1, 
    'space_name': kwargs['space_name'], 
    'space_capacity': kwargs['space_capacity'], 
    'type': kwargs['type'], 
    'updt_date': datetime.now(), 
    }

    space_info_collection.insert_many([space_info])
    print('Space info inserted to mongodb. ')


In [15]:
space_info_example = {
    'space_name': 'B01', 
    'space_capacity': '51-100', 
    'type': 'outdoor', 
}

build_space(**space_info_example)

Space info inserted to mongodb. 


### 用space去篩活動

In [42]:
filter_dict = {}

capacity = '11-20'
if capacity:
    # 要怎麼用column of space data去篩column of event data??
    space_name_lst = space_info_collection.find({'space_capacity': {'$eq': capacity}}).distinct('space_name')
    print(space_name_lst)
    filter_dict['event_location'] = {'$in': space_name_lst}
    print(filter_dict)
else:
    capacity = None

['A01', 'A05', 'B02']
{'event_location': {'$in': ['A01', 'A05', 'B02']}}


#### event_site , event_tag

In [9]:
# event_tag
event_tag_collection = db['event_tag']

In [10]:
if event_tag_collection.find({'tag': {'$eq': 'meeting'}}).distinct('event_id'):
    print('a')

In [28]:
event_name = 'KingN piano concert 2024 3/28 @TP'
event_bgn_time = datetime.fromisoformat('2024-03-28T13:00:00') 
event_end_time = datetime.fromisoformat('2024-03-28T17:00:00') 
event_location = 'C02'
event_host = 's711033123@gm.ntpu.edu.tw'
event_tag = ['concert', 'piano', 'KingN']

event_info = {
    'event_name': event_name, 
    'event_bgn_time': event_bgn_time, 
    'event_end_time': event_end_time, 
    'event_location': event_location, 
    'event_host': event_host, 
    'tag': event_tag
}

build_event(**event_info)

Event created: https://www.google.com/calendar/event?eid=M2kyY2ZiaGV1MW9rNG5rb2pncDZ1cGdsNWcgNjJjMmQzZTgzMTUzOTMxZTY2YzNiOTFiM2VkY2VjYWQ4OGViOGZhMmMwZGY2ZjhhMWZhNzRkNmQ3NjVjMTZiZUBn ,  id: 3i2cfbheu1ok4nkojgp6upgl5g
Event inserted to mongodb. 


In [29]:
event_name = 'KingN piano concert 2024 3/29 @TP'
event_bgn_time = datetime.fromisoformat('2024-03-29T17:00:00') 
event_end_time = datetime.fromisoformat('2024-03-29T21:00:00') 
event_location = 'C02'
event_host = 's711033123@gm.ntpu.edu.tw'
event_tag = ['concert', 'piano', 'KingN']

event_info = {
    'event_name': event_name, 
    'event_bgn_time': event_bgn_time, 
    'event_end_time': event_end_time, 
    'event_location': event_location, 
    'event_host': event_host, 
    'tag': event_tag
}

build_event(**event_info)

Event created: https://www.google.com/calendar/event?eid=NjFpcHNsczV2dWE5c2huZjRrdmN0dHRqZ2MgNjJjMmQzZTgzMTUzOTMxZTY2YzNiOTFiM2VkY2VjYWQ4OGViOGZhMmMwZGY2ZjhhMWZhNzRkNmQ3NjVjMTZiZUBn ,  id: 61ipsls5vua9shnf4kvctttjgc
Event inserted to mongodb. 


In [30]:
event_name = 'KingN piano concert 2024 3/30 @TP - Finale'
event_bgn_time = datetime.fromisoformat('2024-03-30T17:00:00') 
event_end_time = datetime.fromisoformat('2024-03-30T21:00:00') 
event_location = 'B01'
event_host = 's711033123@gm.ntpu.edu.tw'
event_tag = ['concert', 'piano', 'KingN', 'weekend']

event_info = {
    'event_name': event_name, 
    'event_bgn_time': event_bgn_time, 
    'event_end_time': event_end_time, 
    'event_location': event_location, 
    'event_host': event_host, 
    'tag': event_tag
}

build_event(**event_info)

Event created: https://www.google.com/calendar/event?eid=ZzAwbzVzbzI4ZG4zODRndGRtdGFvdm1kcXMgNjJjMmQzZTgzMTUzOTMxZTY2YzNiOTFiM2VkY2VjYWQ4OGViOGZhMmMwZGY2ZjhhMWZhNzRkNmQ3NjVjMTZiZUBn ,  id: g00o5so28dn384gtdmtaovmdqs
Event inserted to mongodb. 
